In [ ]:
from typing import Callable, List, Dict
from sklearn.model_selection import ParameterGrid
import torch
from pprint import pprint

from src.utils import DeviceManager
from src.interleavers import FixedPermuteInterleaver

from src.encoders import *

MANAGER = DeviceManager(seed=1234)

In [ ]:
def generate_interleaver(block_len: int, interleaver_base_seed: int):
    permutation = torch.randperm(
        block_len,
        generator=torch.Generator(device="cpu").manual_seed(interleaver_base_seed),
        device="cpu",
    ).to(MANAGER.device)
    interleaver = FixedPermuteInterleaver(
        input_size=block_len, device_manager=MANAGER, permutation=permutation
    )
    return interleaver

def get_tree_width_report(
    block_lens: List[int], 
    encoder_factories: Dict[str, Callable[[int, DeviceManager, FixedPermuteInterleaver], Encoder]], 
    elimination_tries=300, 
    interleaver_base_seed=108493
):
    run_settings = list(ParameterGrid({"encoder": list(encoder_factories.keys()), "block_len": block_lens}))
    results = []
    for i, run_params in enumerate(run_settings):
        print(f"Run {i+1}/{len(run_settings)}")
        encoder_name = run_params["encoder"]
        encoder_factory = encoder_factories[encoder_name]
        block_len = run_params["block_len"]
        interleaver = generate_interleaver(block_len=block_len, interleaver_base_seed=interleaver_base_seed)
        encoder = encoder_factory(block_len, device_manager=MANAGER, interleaver=interleaver)
        res = encoder.dependency_graph().with_elimination_ordering(sample_thresh=3, tries=elimination_tries).factor_width
        results.append({"encoder": encoder_name, "block_len": block_len, "factor_width": res})
    return sorted(results, key=lambda d: (d["encoder"], d["block_len"]))
    

In [ ]:
block_lens = [5, 8, 11, 14, 20, 30, 40, 50, 60, 70, 80]
encoders = {
    turboae_binary_exact_nobd.__name__: turboae_binary_exact_nobd
}
report_tae = get_tree_width_report(block_lens=block_lens, encoder_factories=encoders)
pprint(report_tae)

In [ ]:
block_lens = [5, 8, 11, 14, 20, 30, 40, 50, 60, 70, 80]
encoders = {
    turbo_random1_nonsys.__name__: turbo_random1_nonsys,
    turbo_random2_nonsys.__name__: turbo_random2_nonsys,
    turbo_random3_nonsys.__name__: turbo_random3_nonsys,
    turbo_random4_nonsys.__name__: turbo_random4_nonsys,
    turbo_random5_nonsys.__name__: turbo_random5_nonsys
}
report = get_tree_width_report(block_lens=block_lens, encoder_factories=encoders)
pprint(report)

In [ ]:
block_lens = [5, 8, 11, 14, 20, 30, 40, 50, 60, 70, 80]
encoders = {
    turbo_random1_nonsys.__name__: turbo_random1_nonsys,
    turbo_random2_nonsys.__name__: turbo_random2_nonsys,
    turbo_random3_nonsys.__name__: turbo_random3_nonsys,
    turbo_random4_nonsys.__name__: turbo_random4_nonsys,
    turbo_random5_nonsys.__name__: turbo_random5_nonsys
}
report = get_tree_width_report(block_lens=block_lens, encoder_factories=encoders)
pprint(report)

In [ ]:
pprint(report)

In [ ]:
block_lens = [30, 40, 50]
encoders = {
    tae_turbo_approximated_nonsys1.__name__: tae_turbo_approximated_nonsys1,
    tae_turbo_approximated_nonsys2.__name__: tae_turbo_approximated_nonsys2,
    tae_turbo_approximated_nonsys3.__name__: tae_turbo_approximated_nonsys3,
    tae_turbo_approximated_nonsys4.__name__: tae_turbo_approximated_nonsys4,
}
report_approx = get_tree_width_report(block_lens=block_lens, encoder_factories=encoders)
pprint(report_approx)